# Minimum
> Minimal way of managing a **(Directed Acyclic Graph)DAG pipeline**

In [ ]:
# default_exp minimum

In [62]:
from forgebox.config import Config
from datetime import datetime
from uuid import uuid4

In [63]:
def now():return datetime.now().timestamp()

'b3d85105-43aa-4865-a686-9365af4dfd7a'

In [27]:
class Link:
    data = Config
    def __init__(self,func):
        self.ensure_funcs()
        func_name = func.__name__
        self.funcs_[func_name] = func
        self.func = self.funcs_[func_name]
        
    def __repr__(self,):
        return f"[Link Function]:{self.func}"
    
    def ensure_funcs(self,):
        if hasattr(self,"funcs_")==False:
            self.__class__.funcs_ = Config()
            
    def ensure_uuid(self,uuid):
        if uuid == False:
            uuid = str(uuid4())
            self.data[uuid] = Config()
        
    def __call__(self,*args,**kwargs, uuid = False):
        print(args)
        print(kwargs)
        self.ensure_uuid(uuid)
        self.data[uuid].update(kwargs)
        rt = self.func(*args,**kwargs)
        print(rt)
        return rt

In [28]:
@Link
def abc(a = 2):
    return a**2

In [29]:
abc(5)

(5,)
{}
25


25

In [30]:
abc

[Link Function]:<function abc at 0x10f3569e0>